# **Meses de oferta (Month of Supply)**

In [3]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres 
nombres_df_filtrados_alfas = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando son las alfa q
    if nombre.startswith("df_") and "pachuca" in nombre and 'alfa' in nombre    
]
nombres_df_filtrados_finance = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando son los financiamientos
    if nombre.startswith("df_") and "pachuca" in nombre and "financiamientos" in nombre    
]


## Oferta de propiedades/Creditos, en dicho mes

In [4]:
meses_financiamientos = pd.concat([globals()[nombre] for nombre in nombres_df_filtrados_finance]).groupby(['mes', 'año']).size().reset_index(name='creditos')
# Filtrar registros con año diferente de 0
meses_financiamientos = meses_financiamientos[meses_financiamientos['año'] != 0]
meses_financiamientos

,mes,año,creditos
0,1,2019,78
1,1,2020,88
2,1,2021,81
3,1,2022,102
4,1,2023,99
...,...,...,...
62,12,2019,142
63,12,2020,132
64,12,2021,137
65,12,2022,160


In [5]:
meses_financiamientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   mes       67 non-null     int64
 1   año       67 non-null     int64
 2   creditos  67 non-null     int64
dtypes: int64(3)
memory usage: 1.7 KB


In [6]:
# Mapeo de meses en español a números
meses_map = {
    'enero': '01', 'febrero': '02', 'feb': '02', 'marzo': '03', 'mar': '03', 'abril': '04',
    'mayo': '05','may': '05', 'junio': '06','jun': '06', 'julio': '07', 'jul': '07', 'agosto': '08','ago': '08',
    'septiembre': '09', 'sep': '09', 'octubre': '10', 'oct': '10', 'noviembre': '11','nov': '11', 'diciembre': '12'
}

In [7]:
# DataFrame para almacenar resultados
meses_alfas = pd.DataFrame(columns=['mes', 'año', 'propiedades'])

# Iterar sobre los nombres de DataFrames
for df_name in nombres_df_filtrados_alfas:
    # Separar los componentes del nombre (mes y año están entre guiones bajos)
    parts = df_name.split('_')
    
    # Verificar si hay un 'q' en la segunda posición para ajustar la posición del mes
    if parts[2] == 'q':
        mes_palabra = parts[3]  # Mes en palabras
        año = parts[4]          # Año
    else:
        mes_palabra = parts[2]  # Mes en palabras
        año = parts[3]          # Año
    
    # Convertir el mes a número usando el diccionario
    mes_numero = meses_map.get(mes_palabra.lower())
    
    if mes_numero:
        # Asumir que el DataFrame real existe en el espacio de trabajo con el nombre correspondiente
        df_real = globals()[df_name]
        
        # Contar registros
        propiedades = len(df_real)
        
        # Agregar los datos al DataFrame final
        nuevo_registro = pd.DataFrame({'mes': [mes_numero], 'año': [año], 'propiedades': [propiedades]})
        meses_alfas = pd.concat([meses_alfas, nuevo_registro], ignore_index=True)
meses_alfas

,mes,año,propiedades
0,04,2024,1547
1,08,2024,1406
2,02,2024,1896
3,07,2024,1763
4,06,2024,1625
5,03,2024,1607
6,05,2024,1806
7,02,2023,928
8,07,2023,165
9,06,2023,1825


In [8]:
meses_alfas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   mes          14 non-null     object
 1   año          14 non-null     object
 2   propiedades  14 non-null     object
dtypes: object(3)
memory usage: 468.0+ bytes


In [9]:
# Convertir las columnas 'mes' y 'año' a int, asegurándonos que no haya valores no convertibles
meses_alfas['mes'] = pd.to_numeric(meses_alfas['mes'], errors='coerce').fillna(0).astype(int)
meses_alfas['año'] = pd.to_numeric(meses_alfas['año'], errors='coerce').fillna(0).astype(int)
meses_alfas['propiedades'] = pd.to_numeric(meses_alfas['propiedades'], errors='coerce').fillna(0).astype(int)

In [10]:
meses_alfas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   mes          14 non-null     int32
 1   año          14 non-null     int32
 2   propiedades  14 non-null     int32
dtypes: int32(3)
memory usage: 300.0 bytes


In [11]:
df_merged = pd.merge(meses_alfas, meses_financiamientos, on=['mes', 'año'], how='inner')

# Reemplazar valores nulos con ceros en las columnas 'Creditos' y 'Propiedades'
df_merged['creditos'].fillna(0, inplace=True)
df_merged['propiedades'].fillna(0, inplace=True)


# Evitar división entre cero utilizando np.where()
df_merged['resultado'] = df_merged.apply(lambda row: row['propiedades'] / row['creditos'] 
                                         if row['creditos'] > 0 else 0, axis=1)
df_merged['resultado'] = round(df_merged['resultado'],2)
# Filtramos los resultados que tengan valores de 'Creditos' mayor que cero para evitar divisiones entre cero
df_merged = df_merged[df_merged['creditos'] > 0]

# Mostramos el DataFrame resultante con las columnas 'MES', 'AÑO' y 'resultado'
df_resultado = df_merged[['mes', 'año', 'resultado']]
df_resultado


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19252\2191916376.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['creditos'].fillna(0, inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19252\2191916376.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

,mes,año,resultado
0,4,2024,14.32
1,2,2024,18.06
2,7,2024,12.50
3,6,2024,13.21
4,3,2024,13.50
5,5,2024,17.20
6,2,2023,11.32
7,7,2023,1.56
8,6,2023,12.59
9,3,2023,14.11


In [12]:
# Convertir los valores de la columna mes a enteros
df_resultado['mes'] = df_resultado['mes'].astype(int)
# Mapear los números de mes a sus nombres correspondientes
meses_texto = {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}
# Convertir el número de mes en texto
df_resultado['mes'] = df_resultado.apply(lambda row: f"{meses_texto[row['mes']]}_{int(row['año'])}", axis=1)
# Eliminar la columna 'año' ahora que ya no es necesaria
df_resultado = df_resultado.drop(columns=['año'])
df_resultado

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19252\1054691112.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultado['mes'] = df_resultado['mes'].astype(int)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19252\1054691112.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultado['mes'] = df_resultado.apply(lambda row: f"{meses_texto[row['mes']]}_{int(row['año'])}", axis=1)


,mes,resultado
0,Abril_2024,14.32
1,Febrero_2024,18.06
2,Julio_2024,12.50
3,Junio_2024,13.21
4,Marzo_2024,13.50
5,Mayo_2024,17.20
6,Febrero_2023,11.32
7,Julio_2023,1.56
8,Junio_2023,12.59
9,Marzo_2023,14.11


In [13]:
conteo_2023 = df_resultado[df_resultado['mes'].str.contains('2024')]
conteo_2023

,mes,resultado
0,Abril_2024,14.32
1,Febrero_2024,18.06
2,Julio_2024,12.50
3,Junio_2024,13.21
4,Marzo_2024,13.50
5,Mayo_2024,17.20


In [68]:
# Obtener los datos
promedio = conteo_2023['resultado']
meses = conteo_2023['mes']

# Ordenar los meses en el orden deseado
meses_ordenados = ['Febrero_2024', 'Marzo_2024', 'Abril_2024', 'Mayo_2024', 'Junio_2024', 'Julio_2024']
meses = meses[meses.isin(meses_ordenados)]

# Crear figura
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=meses,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for mes, dia in zip(meses, promedio):
    fig.add_annotation(
        x=mes,
        y=dia,
        text=str(dia),  # Texto que se mostrará (el valor del día)
        showarrow=False,  # No mostrar la flecha
        font=dict(color='black', size=12),  # Estilo del texto
        xshift=0,  # Desplazamiento horizontal del texto
        yshift=17,  # Desplazamiento vertical del texto
        textangle=0  # Ángulo del texto
    )
# Actualizar diseño
fig.update_layout(
    #title='Meses de oferta',
    xaxis_title='Mes de oferta créditos',
    yaxis_title='Promedio',
    xaxis=dict(gridcolor='#dddcda', tickmode='array', tickvals=list(range(len(meses))), ticktext=meses),
    yaxis=dict(gridcolor='#dddcda', gridwidth=1),
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)   
)


fig.show()

In [ ]:
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_MonthSupply_2023', carpeta='assets/graficas')

# OLD

In [14]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados_alfas:
    # Seleccionar las columnas 'id' y 'categoria'
    #segment_df = globals()[nombre_df][['año','mes']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(df)
alfas_pachuca = pd.concat(dataframes_list)
alfas_pachuca.columns

NameError: name 'df' is not defined

In [8]:
# Concatenar los DataFrames
fin_pachuca = pd.concat(dataframes_list)

# Filtrar registros con año diferente de 0
fin_pachuca_filtrado = fin_pachuca[fin_pachuca['año'] != 0]

# Contar los registros por año y mes
conteo_registros_por_mes = fin_pachuca_filtrado.groupby(['año', 'mes']).size().reset_index(name='conteo')
conteo_registros_por_mes

,año,mes,conteo
0,2019,1,78
1,2019,2,87
2,2019,3,88
3,2019,4,79
4,2019,5,103
...,...,...,...
62,2024,3,119
63,2024,4,108
64,2024,5,105
65,2024,6,123


In [9]:
conteos_por_año = fin_pachuca.groupby('año').size()

# Filtrar el conteo para cada año específico
conteo_2022 = conteos_por_año.get(2022, 0)  # Si no hay registros para el año 2022, devolverá 0
conteo_2023 = conteos_por_año.get(2023, 0)  # Si no hay registros para el año 2023, devolverá 0
conteo_2024 = conteos_por_año.get(2024, 0)  # Si no hay registros para el año 2025, devolverá 0


print("Conteo de registros para 2022:", conteo_2022)
print("Conteo de registros para 2023:", conteo_2023)
print("Conteo de registros para 2024:", conteo_2024)

Conteo de registros para 2022: 1388
Conteo de registros para 2023: 1399
Conteo de registros para 2024: 792


In [10]:
promedio_2022 = (conteo_2022/12).round(2)
promedio_2023 = (conteo_2023/12).round(2)
promedio_2024 = (conteo_2024/12).round(2)
print("Promedio de créditos en 2022:", promedio_2022)
print("Promedio de créditos en 2023:", promedio_2023)
print("Promedio de créditos en 2024:", promedio_2024)

Promedio de créditos en 2022: 115.67
Promedio de créditos en 2023: 116.58
Promedio de créditos en 2024: 66.0


In [6]:
conteo_registros_por_mes['promedio_creditos'] = conteo_registros_por_mes.apply(
    lambda row: round(row['conteo'] / promedio_2022, 2) if row['año'] == 2022 
    else round(row['conteo'] / promedio_2023, 2) if row['año'] == 2023 
    else round(row['conteo'] / promedio_2024, 2), axis=1
)
conteo_registros_por_mes

,año,mes,conteo,promedio_creditos
0,2019,1,78,1.18
1,2019,2,87,1.32
2,2019,3,88,1.33
3,2019,4,79,1.20
4,2019,5,103,1.56
...,...,...,...,...
62,2024,3,119,1.80
63,2024,4,108,1.64
64,2024,5,105,1.59
65,2024,6,123,1.86


In [7]:
# Convertir los valores de la columna mes a enteros
conteo_registros_por_mes['mes'] = conteo_registros_por_mes['mes'].astype(int)
# Mapear los números de mes a sus nombres correspondientes
meses_texto = {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}
# Convertir el número de mes en texto
conteo_registros_por_mes['mes'] = conteo_registros_por_mes.apply(lambda row: f"{meses_texto[row['mes']]}_{int(row['año'])}", axis=1)
# Eliminar la columna 'año' ahora que ya no es necesaria
conteo_registros_por_mes = conteo_registros_por_mes.drop(columns=['año'])
conteo_registros_por_mes

,mes,conteo,promedio_creditos
0,Enero_2019,78,1.18
1,Febrero_2019,87,1.32
2,Marzo_2019,88,1.33
3,Abril_2019,79,1.20
4,Mayo_2019,103,1.56
...,...,...,...
62,Marzo_2024,119,1.80
63,Abril_2024,108,1.64
64,Mayo_2024,105,1.59
65,Junio_2024,123,1.86


In [8]:
# Filtrar solo los registros correspondientes a 2023
conteo_2023 = conteo_registros_por_mes[conteo_registros_por_mes['mes'].str.contains('2023')]
# Seleccionar las columnas mes, conteo y promedio_creditos
conteo_2023 = conteo_2023[['mes', 'conteo', 'promedio_creditos']]
conteo_2023

,mes,conteo,promedio_creditos
48,Enero_2023,99,0.85
49,Febrero_2023,82,0.70
50,Marzo_2023,126,1.08
51,Abril_2023,95,0.81
52,Mayo_2023,148,1.27
53,Junio_2023,145,1.24
54,Julio_2023,106,0.91
55,Agosto_2023,98,0.84
56,Septiembre_2023,105,0.90
57,Octubre_2023,125,1.07


In [15]:
# Obtener los datos
promedio = conteo_2023['promedio_creditos']
meses = conteo_2023['mes']

# Crear figura
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=meses,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for mes, dia in zip(meses, promedio):
    fig.add_annotation(
        x=mes,
        y=dia,
        text=str(dia),  # Texto que se mostrará (el valor del día)
        showarrow=False,  # No mostrar la flecha
        font=dict(color='black', size=12),  # Estilo del texto
        xshift=0,  # Desplazamiento horizontal del texto
        yshift=17,  # Desplazamiento vertical del texto
        textangle=0  # Ángulo del texto
    )
# Actualizar diseño
fig.update_layout(
    #title='Meses de oferta',
    xaxis_title='Mes de oferta créditos',
    yaxis_title='Promedio',
    xaxis=dict(gridcolor='#dddcda', tickmode='array', tickvals=list(range(len(meses))), ticktext=meses),
    yaxis=dict(gridcolor='#dddcda', gridwidth=1),
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)   
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_MonthSupply_2023', carpeta='assets/graficas')
fig.show()

In [16]:
# Filtrar solo los registros correspondientes a 2023
conteo_2024 = conteo_registros_por_mes[conteo_registros_por_mes['mes'].str.contains('2024')]
# Seleccionar las columnas mes, conteo y promedio_creditos
conteo_2024 = conteo_2024[['mes', 'conteo', 'promedio_creditos']]
conteo_2024

,mes,conteo,promedio_creditos
60,Enero_2024,91,1.38
61,Febrero_2024,105,1.59
62,Marzo_2024,119,1.80
63,Abril_2024,108,1.64
64,Mayo_2024,105,1.59
65,Junio_2024,123,1.86
66,Julio_2024,141,2.14


In [17]:
# Obtener los datos
promedio = conteo_2024['promedio_creditos']
meses = conteo_2024['mes']

# Crear figura
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=meses,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for mes, dia in zip(meses, promedio):
    fig.add_annotation(
        x=mes,
        y=dia,
        text=str(dia),  # Texto que se mostrará (el valor del día)
        showarrow=False,  # No mostrar la flecha
        font=dict(color='black', size=12),  # Estilo del texto
        xshift=0,  # Desplazamiento horizontal del texto
        yshift=17,  # Desplazamiento vertical del texto
        textangle=0  # Ángulo del texto
    )
# Actualizar diseño
fig.update_layout(
    #title='Meses de oferta',
    xaxis_title='Mes de oferta créditos',
    yaxis_title='Promedio',
    xaxis=dict(gridcolor='#dddcda', tickmode='array', tickvals=list(range(len(meses))), ticktext=meses),
    yaxis=dict(gridcolor='#dddcda', gridwidth=1),
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)   
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_MonthSupply_2024', carpeta='assets/graficas')
fig.show()

In [11]:
# Filtrar solo los registros correspondientes a 2023
#conteo_2024 = conteo_registros_por_mes
conteo_2024 = conteo_registros_por_mes[conteo_registros_por_mes['mes'].str.contains('2023|2024')]

# Seleccionar las columnas mes, conteo y promedio_creditos
conteo_2024 = conteo_2024[['mes', 'conteo', 'promedio_creditos']]
conteo_2024

,mes,conteo,promedio_creditos
48,Enero_2023,99,0.85
49,Febrero_2023,82,0.70
50,Marzo_2023,126,1.08
51,Abril_2023,95,0.81
52,Mayo_2023,148,1.27
53,Junio_2023,145,1.24
54,Julio_2023,106,0.91
55,Agosto_2023,98,0.84
56,Septiembre_2023,105,0.90
57,Octubre_2023,125,1.07


In [12]:
# Obtener los datos
promedio = conteo_2024['promedio_creditos']
meses = conteo_2024['mes']

# Crear figura
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=meses,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for mes, dia in zip(meses, promedio):
    fig.add_annotation(
        x=mes,
        y=dia,
        text=str(dia),  # Texto que se mostrará (el valor del día)
        showarrow=False,  # No mostrar la flecha
        font=dict(color='black', size=12),  # Estilo del texto
        xshift=0,  # Desplazamiento horizontal del texto
        yshift=17,  # Desplazamiento vertical del texto
        textangle=0  # Ángulo del texto
    )
# Actualizar diseño
fig.update_layout(
    #title='Meses de oferta',
    xaxis_title='Mes de oferta créditos',
    yaxis_title='Promedio',
    xaxis=dict(gridcolor='#dddcda', tickmode='array', tickvals=list(range(len(meses))), ticktext=meses),
    yaxis=dict(gridcolor='#dddcda', gridwidth=1),
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)   
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_MonthSupply_years_2324', carpeta='assets/graficas')
fig.show()

In [31]:
# Filtrar solo los registros correspondientes a 2023
conteo_all = conteo_registros_por_mes
# Seleccionar las columnas mes, conteo y promedio_creditos
conteo_all = conteo_all[['mes', 'conteo', 'promedio_creditos']]
conteo_all

,mes,conteo,promedio_creditos
0,Enero_2019,78,0.77
1,Febrero_2019,87,0.86
2,Marzo_2019,88,0.87
3,Abril_2019,79,0.78
4,Mayo_2019,103,1.02
...,...,...,...
62,Marzo_2024,119,1.05
63,Abril_2024,108,0.95
64,Mayo_2024,105,0.93
65,Junio_2024,123,1.09


In [34]:
conteo_registros_por_mes['año'] = conteo_registros_por_mes['mes'].apply(lambda x: int(x.split('_')[1]))
promedio_por_año = conteo_registros_por_mes.groupby('año')['conteo'].mean().round(2)
conteo_anual = conteo_registros_por_mes.groupby('año')['conteo'].sum().reset_index()
conteo_anual['promedio_creditos'] = conteo_anual.apply(
    lambda row: round(row['conteo'] / promedio_por_año[row['año']], 2), axis=1
)
conteo_all = conteo_anual[['año', 'conteo', 'promedio_creditos']]
conteo_all

,año,conteo,promedio_creditos
0,2019,1215,12.0
1,2020,1178,12.0
2,2021,1430,12.0
3,2022,1388,12.0
4,2023,1399,12.0
5,2024,792,7.0


In [33]:
# Obtener los datos
promedio = conteo_all['promedio_creditos']
meses = conteo_all['año']

# Crear figura
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=meses,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for mes, dia in zip(meses, promedio):
    fig.add_annotation(
        x=mes,
        y=dia,
        text=str(dia),  # Texto que se mostrará (el valor del día)
        showarrow=False,  # No mostrar la flecha
        font=dict(color='black', size=12),  # Estilo del texto
        xshift=0,  # Desplazamiento horizontal del texto
        yshift=17,  # Desplazamiento vertical del texto
        textangle=0  # Ángulo del texto
    )
# Actualizar diseño
fig.update_layout(
    #title='Meses de oferta',
    xaxis_title='Año de oferta créditos',
    yaxis_title='Promedio',
    xaxis=dict(gridcolor='#dddcda', tickmode='array', tickvals=list(range(len(meses))), ticktext=meses),
    yaxis=dict(gridcolor='#dddcda', gridwidth=1),
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)   
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_MonthSupply', carpeta='assets/graficas')
fig.show()